In [332]:
import pandas as pd
import numpy as np
import math
import os

In [333]:
price_data =  pd.read_hdf(r'C:\Users\Anonymous\Documents\Data Analysis\Backtesting - Final\Final\Backtesting\FullOHLCFinal11.h5')  

In [334]:
close_df = price_data
close_df = close_df.drop(columns=['Open', 'High', 'Low', 'Volume', 'VolumeCcy', 'Returns', 'Log Returns'])

In [335]:
returns_df = (close_df.groupby(['Timestamp', 'Token'])['Close']
    .first()
    .unstack())

returns_df = returns_df.sort_index(ascending=True)
returns_df = returns_df.pct_change(1)

In [336]:
returns_df

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1633414920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1633414980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1633415040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1633415100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1633415160000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-0.002637,-0.001304,-0.000735,-0.000490,-0.004329,-0.000176,0.0,0.000881,-0.001053,0.0,...,-0.000655,-0.001112,0.0,-0.006161,-0.001826,-0.000804,-0.002617,-0.002273,-0.000221,0.000000
1649026680000,-0.001322,-0.000082,-0.002206,0.000526,0.002899,-0.000143,0.0,-0.000880,-0.001756,0.0,...,-0.001248,-0.000159,0.0,-0.000698,-0.000444,-0.000804,-0.001705,0.000000,0.002321,0.000000
1649026740000,-0.001324,-0.000980,0.000000,0.000400,0.000000,0.000017,0.0,0.000000,0.000880,0.0,...,0.001468,0.000318,0.0,-0.000671,-0.000166,-0.001207,0.001051,0.000759,-0.001213,-0.001366


In [337]:
returns_df = returns_df.drop(columns=['TUSDUSDT.BINANCE', 'USTUSDT.BINANCE', 'USDCUSDT.BINANCE', 'UTKUSDT.BINANCE', 'DAIUSDT.BINANCE'])

In [338]:
returns_df = returns_df.tail(60*24*14)

Now that we have the returns matrix, we need to standardize the data. This is because PCA will overweight based on the magnitude of the returns here. To standardize, we should do this on a rolling basis to prevent lookahead being introduced.

In [339]:
standardized = (returns_df - returns_df.expanding(120).mean()) / returns_df.expanding(120).std()

In [340]:
standardized

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1647817320000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-1.843675,-0.746210,-0.529308,-0.173349,-2.454993,-0.152590,-0.005359,0.515445,-0.456258,-0.004219,...,-0.354132,-0.332327,0.000237,-2.111198,-1.250004,-0.555365,-0.702968,-1.156763,-0.123111,-0.002364
1649026680000,-0.928473,-0.059661,-1.580536,0.170426,1.637662,-0.125892,-0.005359,-0.523369,-0.757934,-0.004219,...,-0.669392,-0.044438,0.000237,-0.242748,-0.305067,-0.555787,-0.464499,-0.007672,1.142239,-0.002364
1649026740000,-0.929648,-0.563870,-0.004125,0.127874,-0.003654,0.001033,-0.005359,-0.004173,0.372344,-0.004219,...,0.774197,0.099709,0.000237,-0.233724,-0.115667,-0.830739,0.256733,0.376230,-0.616937,-0.493905


In [341]:
covariance_df = standardized.cov()
correlation_df = standardized.corr()

In [342]:
covariance_df

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Token,,,,,,,,,,,,,,,,,,,,,
1INCHUSDT.BINANCE,1.334515,0.618123,0.016257,0.061790,0.021047,0.557871,0.005740,0.026784,0.286580,0.030877,...,0.409651,0.019706,-0.011814,0.001264,0.464034,0.588253,0.334292,0.023529,0.465465,-0.014215
AAVEUSDT.BINANCE,0.618123,1.372285,-0.001983,0.063151,0.020680,0.502789,-0.009292,0.014762,0.280051,0.011913,...,0.348612,0.017458,-0.006511,0.000178,0.410562,0.511363,0.300494,0.021252,0.425303,-0.000892
ABTUSDT.BINANCE,0.016257,-0.001983,1.178267,0.010172,-0.007801,0.015191,0.009667,-0.001666,0.000953,0.001225,...,0.016063,0.010578,-0.002014,0.005182,0.015217,0.009258,-0.018489,-0.000199,-0.015940,0.009228
ACAUSDT.BINANCE,0.061790,0.063151,0.010172,1.433579,0.028183,0.062745,0.002131,-0.007043,0.046769,0.024515,...,0.069128,0.007233,0.001304,0.011359,0.048132,0.061447,0.041045,0.035153,0.049495,0.021851
ACTUSDT.BINANCE,0.021047,0.020680,-0.007801,0.028183,1.276001,0.018393,0.004053,0.007616,0.026266,-0.014656,...,0.042436,-0.013733,0.010467,0.006606,0.012841,0.024771,0.004439,0.001843,0.005539,-0.004601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZENUSDT.BINANCE,0.588253,0.511363,0.009258,0.061447,0.024771,0.487261,-0.012459,0.028209,0.258266,0.015787,...,0.374767,0.019075,-0.007642,0.005310,0.443909,1.022604,0.301804,0.009663,0.448532,-0.008506
ZILUSDT.BINANCE,0.334292,0.300494,-0.018489,0.041045,0.004439,0.327040,-0.015547,0.009517,0.174221,0.009813,...,0.213481,0.000617,-0.000601,0.015185,0.239723,0.301804,2.068110,0.000169,0.298670,0.000942
ZKSUSDT.BINANCE,0.023529,0.021252,-0.000199,0.035153,0.001843,0.010240,0.009213,0.000915,0.013028,-0.002722,...,0.003145,0.010543,0.005585,0.001998,0.001308,0.009663,0.000169,1.554823,0.016693,0.008334


In [343]:
correlation_df

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Token,,,,,,,,,,,,,,,,,,,,,
1INCHUSDT.BINANCE,1.000000,0.456763,0.012964,0.044673,0.016129,0.489307,0.003910,0.020724,0.242072,0.026442,...,0.282425,0.019298,-0.012398,0.001074,0.410042,0.503557,0.201223,0.016335,0.306453,-0.010963
AAVEUSDT.BINANCE,0.456763,1.000000,-0.001560,0.045025,0.015628,0.434884,-0.006241,0.011263,0.233278,0.010061,...,0.237012,0.016860,-0.006738,0.000149,0.357764,0.431671,0.178372,0.014549,0.276131,-0.000678
ABTUSDT.BINANCE,0.012964,-0.001560,1.000000,0.007827,-0.006362,0.014180,0.007007,-0.001372,0.000856,0.001117,...,0.011785,0.011025,-0.002250,0.004684,0.014311,0.008434,-0.011844,-0.000147,-0.011169,0.007574
ACAUSDT.BINANCE,0.044673,0.045025,0.007827,1.000000,0.020838,0.053098,0.001400,-0.005258,0.038116,0.020256,...,0.045983,0.006834,0.001320,0.009308,0.041036,0.050750,0.023838,0.023546,0.031440,0.016259
ACTUSDT.BINANCE,0.016129,0.015628,-0.006362,0.020838,1.000000,0.016499,0.002823,0.006026,0.022690,-0.012835,...,0.029920,-0.013754,0.011234,0.005738,0.011604,0.021685,0.002733,0.001308,0.003730,-0.003628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZENUSDT.BINANCE,0.503557,0.431671,0.008434,0.050750,0.021685,0.488222,-0.009694,0.024934,0.249214,0.015444,...,0.295161,0.021339,-0.009162,0.005152,0.448105,1.000000,0.207532,0.007664,0.337348,-0.007494
ZILUSDT.BINANCE,0.201223,0.178372,-0.011844,0.023838,0.002733,0.230422,-0.008506,0.005915,0.118215,0.006750,...,0.118229,0.000485,-0.000506,0.010360,0.170162,0.207532,1.000000,0.000094,0.157958,0.000583
ZKSUSDT.BINANCE,0.016335,0.014549,-0.000147,0.023546,0.001308,0.008321,0.005813,0.000656,0.010195,-0.002160,...,0.002009,0.009566,0.005430,0.001572,0.001070,0.007664,0.000094,1.000000,0.010182,0.005954


In [344]:
corr_results = correlation_df.mean().sort_values(axis=0, ascending=False)
corr_results

Token
MATICUSDT.BINANCE    0.206906
BTCUSDT.BINANCE      0.205328
ETHUSDT.BINANCE      0.204301
LINKUSDT.BINANCE     0.203125
LTCUSDT.BINANCE      0.195118
                       ...   
XPOUSDT.BINANCE      0.000480
MCOUSDT.BINANCE      0.000407
ETMUSDT.BINANCE      0.000116
BHPUSDT.BINANCE     -0.000874
BORAUSDT.BINANCE    -0.004893
Length: 338, dtype: float64

In [345]:
corr_results.mean()*100

6.548981816869641

In [346]:
import sklearn
from sklearn.decomposition import PCA

In [347]:
pca = PCA()
pca.fit(covariance_df)

PCA()

In [348]:
total_var_explained = pd.Series(pca.explained_variance_ratio_).to_frame('Explained Variance').head(5).style.format('{:,.2%}'.format)
total_var_explained

,Explained Variance
0,78.42%
1,0.55%
2,0.39%
3,0.38%
4,0.32%


In [349]:
weight =  pd.DataFrame(pca.components_)

In [350]:
weight.columns = correlation_df.index

In [351]:
weight

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
0,0.108503,0.097908,0.000017,0.010846,0.003049,0.093648,-0.000835,0.002511,0.049844,0.002873,...,0.071518,0.002398,-0.002578,-0.000164,0.077552,0.095779,0.060925,0.001403,0.081604,-0.002282
1,-0.053373,-0.070667,0.014561,0.070492,0.011393,-0.072633,0.022243,0.018846,0.034551,0.013929,...,0.102057,-0.008176,0.004736,-0.007547,-0.035256,-0.037296,-0.061264,0.025005,0.010623,-0.010057
2,-0.002832,0.010515,-0.007630,-0.004234,-0.051055,-0.002957,0.001076,0.001643,0.003093,-0.002621,...,0.003716,0.000981,0.004173,-0.005336,-0.001118,-0.000517,0.005352,-0.010792,-0.001812,0.008778
3,-0.007765,-0.006208,-0.000223,0.006707,-0.029815,-0.011693,-0.013540,0.000487,-0.007624,-0.008554,...,0.014344,-0.007854,-0.006891,0.001791,-0.009097,0.000159,0.010698,0.002133,0.007738,-0.025969
4,-0.007689,0.007666,-0.006083,-0.029632,-0.012551,0.007571,-0.020998,-0.011252,0.000018,-0.002569,...,0.002046,-0.012141,-0.026329,0.004912,-0.011024,-0.010942,-0.021807,-0.001277,-0.007065,0.002958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,0.002582,0.009525,-0.007835,0.002203,-0.004506,0.030775,-0.001119,0.007864,-0.014393,0.002099,...,0.010574,-0.001266,-0.018284,0.000109,-0.025430,0.021873,-0.002156,-0.002078,-0.015614,-0.019523
334,-0.002117,-0.032283,0.001838,0.002664,0.001221,-0.003037,0.000828,0.000764,-0.009527,0.006148,...,0.012764,-0.000343,0.008940,-0.000608,-0.002281,0.022042,0.002989,-0.003497,-0.003332,0.000173
335,-0.000599,-0.013751,-0.026430,-0.010340,-0.023169,-0.010899,-0.017258,-0.026484,-0.015685,-0.034190,...,-0.003679,-0.060401,-0.047157,-0.036429,-0.005550,0.012662,-0.008449,-0.016994,-0.008754,-0.037809
336,-0.001716,0.011393,-0.005336,0.002488,-0.005208,-0.020629,-0.004903,0.001736,0.010588,0.000748,...,0.007047,-0.007834,-0.007709,-0.004276,-0.003935,0.001682,0.000783,-0.000244,-0.000401,-0.007025


In [352]:
q = weight / returns_df.std() # WHY? q = prin / std

In [353]:
q

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
0,75.517707,54.979286,0.012285,3.671137,1.726608,74.461936,-0.310597,1.481337,21.375701,0.539319,...,38.015438,0.724575,-0.948756,-0.056070,53.000957,65.339890,15.941392,0.709582,40.623300,-0.821019
1,-37.147435,-39.682379,10.406394,23.859588,6.451703,-57.752670,8.271843,11.116160,14.817231,2.615075,...,54.248788,-2.470288,1.743123,-2.581263,-24.095063,-25.443000,-16.029954,12.643870,5.288120,-3.618950
2,-1.971090,5.904773,-5.453137,-1.433255,-28.912511,-2.350867,0.400268,0.968964,1.326576,-0.492041,...,1.975354,0.296315,1.535952,-1.824950,-0.763980,-0.352622,1.400441,-5.457152,-0.901788,3.158758
3,-5.404282,-3.485843,-0.159640,2.270267,-16.883965,-9.297485,-5.035404,0.287336,-3.269442,-1.605954,...,7.624624,-2.372780,-2.536243,0.612632,-6.216824,0.108646,2.799310,1.078369,3.851920,-9.344722
4,-5.351310,4.304709,-4.347606,-10.029675,-7.107578,6.019675,-7.808932,-6.636496,0.007710,-0.482300,...,1.087445,-3.668005,-9.689977,1.680223,-7.534356,-7.464321,-5.705896,-0.645809,-3.517114,1.064320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,1.796903,5.348758,-5.599400,0.745610,-2.551831,24.469931,-0.416216,4.638520,-6.172548,0.394104,...,5.620452,-0.382367,-6.729246,0.037255,-17.379491,14.921514,-0.564213,-1.050945,-7.772719,-7.025122
334,-1.473423,-18.128184,1.313490,0.901706,0.691278,-2.414797,0.307831,0.450730,-4.085466,1.154292,...,6.784800,-0.103772,3.290339,-0.207866,-1.559105,15.037121,0.782105,-1.768438,-1.658864,0.062128
335,-0.416889,-7.721992,-18.889616,-3.499682,-13.120580,-8.666301,-6.417999,-15.620932,-6.726574,-6.418727,...,-1.955365,-18.248817,-17.355899,-12.459889,-3.793142,8.637818,-2.210721,-8.593085,-4.357832,-13.605308
336,-1.194003,6.397458,-3.813563,0.842064,-2.949303,-16.402631,-1.823457,1.023876,4.540743,0.140447,...,3.746056,-2.366848,-2.837093,-1.462378,-2.689538,1.147430,0.204949,-0.123486,-0.199789,-2.528039


In [354]:
factor_returns = pd.DataFrame(np.dot(returns_df, q.transpose()),index=returns_df.index)

In [355]:
factor_returns.columns = correlation_df.index

In [356]:
factor_returns

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1647817320000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647817560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-9.262077,-5.092229,0.708397,-0.596344,-0.273587,-0.190707,-1.629857,0.533970,0.055740,1.787072,...,0.434634,-0.978691,-0.160472,0.164884,0.401318,-0.204149,-1.347879,0.496339,0.933428,0.970493
1649026680000,-5.335865,-2.063322,13.924322,-4.287383,0.284975,-0.209963,-0.743329,0.599849,0.270858,-0.118683,...,-1.662213,-1.442936,-0.269837,-0.988374,-0.589467,-0.043235,1.114367,-0.217926,-0.254282,-0.379896
1649026740000,-1.062158,-0.832066,-7.508912,-5.741529,1.054498,-0.391677,0.406921,-0.409403,-0.457628,0.110056,...,0.284084,0.096267,-0.102323,0.307194,-0.412491,0.325923,-0.111996,0.786211,-0.448410,0.999308


In [357]:
from sklearn.linear_model import LinearRegression

In [358]:
returns_df.replace([np.inf, -np.inf], 0, inplace=True)
factor_returns.replace([np.inf, -np.inf], 0, inplace=True)

In [359]:
returns_df

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1647817320000,-0.001083,0.000325,0.000000,0.000043,0.000000,-0.001232,0.0,0.000000,-0.000976,0.000000,...,-0.000304,0.000112,0.002398,0.001319,-0.001093,-0.000486,0.000465,0.000000,-0.000180,0.000000
1647817380000,-0.000128,0.001039,0.000000,-0.000438,0.000000,-0.000362,0.0,0.000000,0.000698,0.012987,...,-0.001368,0.000672,-0.019139,0.000125,0.000403,-0.000243,-0.000232,0.000000,-0.000180,0.000000
1647817440000,-0.000829,0.000259,0.000000,0.000679,0.001595,0.000147,0.0,0.000000,0.001395,0.000000,...,-0.000190,0.001007,-0.009756,-0.001066,-0.000978,0.000000,0.000000,0.000000,-0.000180,0.000000
1647817500000,-0.000766,0.000583,0.000000,-0.001254,-0.001592,-0.001087,0.0,0.000000,0.000209,0.000000,...,0.000000,0.001229,0.004926,0.004520,-0.001037,-0.000487,-0.001394,-0.001982,0.000361,0.000000
1647817560000,0.000894,0.000194,0.000000,-0.000361,0.000000,-0.000181,0.0,0.000000,0.000139,0.000000,...,-0.000114,0.007256,-0.004902,0.000687,0.000404,-0.000730,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-0.002637,-0.001304,-0.000735,-0.000490,-0.004329,-0.000176,0.0,0.000881,-0.001053,0.000000,...,-0.000655,-0.001112,0.000000,-0.006161,-0.001826,-0.000804,-0.002617,-0.002273,-0.000221,0.000000
1649026680000,-0.001322,-0.000082,-0.002206,0.000526,0.002899,-0.000143,0.0,-0.000880,-0.001756,0.000000,...,-0.001248,-0.000159,0.000000,-0.000698,-0.000444,-0.000804,-0.001705,0.000000,0.002321,0.000000
1649026740000,-0.001324,-0.000980,0.000000,0.000400,0.000000,0.000017,0.0,0.000000,0.000880,0.000000,...,0.001468,0.000318,0.000000,-0.000671,-0.000166,-0.001207,0.001051,0.000759,-0.001213,-0.001366


In [360]:
returns_df = returns_df.fillna(0)
factor_returns = factor_returns.fillna(0)



In [361]:
df1 = returns_df[returns_df.isna().any(axis=1)]
df1

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,


In [362]:
residual = pd.DataFrame(columns = returns_df.columns, index = returns_df.index)
coef = pd.DataFrame(columns = returns_df.columns, index = range(338))
ols = LinearRegression()
for i in returns_df.columns:
    ols.fit(factor_returns, returns_df[i])
    residual[i] = returns_df[i]-ols.intercept_-np.dot(factor_returns, ols.coef_)
    coef[i] = ols.coef_

In [363]:
residual

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1647817320000,-0.001089,0.000304,-0.000005,0.000023,-0.000003,-0.001246,-0.000012,-0.000005,-0.000987,-0.000016,...,-0.000312,0.000124,0.002401,0.001311,-0.001095,-0.000496,0.000378,-0.000009,-0.000200,-0.000007
1647817380000,-0.000134,0.001018,-0.000005,-0.000458,-0.000003,-0.000376,-0.000012,-0.000005,0.000687,0.012971,...,-0.001376,0.000684,-0.019135,0.000118,0.000401,-0.000253,-0.000319,-0.000009,-0.000200,-0.000007
1647817440000,-0.000835,0.000239,-0.000005,0.000659,0.001592,0.000133,-0.000012,-0.000005,0.001384,-0.000016,...,-0.000198,0.001019,-0.009753,-0.001074,-0.000980,-0.000009,-0.000087,-0.000009,-0.000200,-0.000007
1647817500000,-0.000772,0.000563,-0.000005,-0.001273,-0.001595,-0.001101,-0.000012,-0.000005,0.000198,-0.000016,...,-0.000008,0.001242,0.004929,0.004512,-0.001039,-0.000496,-0.001481,-0.001992,0.000341,-0.000007
1647817560000,0.000888,0.000174,-0.000005,-0.000381,-0.000003,-0.000195,-0.000012,-0.000005,0.000128,-0.000016,...,-0.000122,0.007268,-0.004899,0.000679,0.000401,-0.000740,-0.000087,-0.000009,-0.000020,-0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-0.000008,-0.000025,-0.000007,-0.000024,-0.000004,-0.000017,-0.000014,-0.000006,-0.000014,-0.000020,...,-0.000010,0.000015,0.000004,-0.000010,-0.000003,-0.000011,-0.000106,-0.000012,-0.000024,-0.000008
1649026680000,-0.000009,-0.000028,-0.000007,-0.000027,-0.000004,-0.000019,-0.000016,-0.000007,-0.000015,-0.000022,...,-0.000011,0.000017,0.000004,-0.000011,-0.000003,-0.000013,-0.000119,-0.000013,-0.000027,-0.000009
1649026740000,-0.000007,-0.000022,-0.000006,-0.000021,-0.000003,-0.000015,-0.000013,-0.000006,-0.000012,-0.000018,...,-0.000009,0.000013,0.000004,-0.000009,-0.000002,-0.000010,-0.000094,-0.000010,-0.000022,-0.000008


In [364]:
coef

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
0,0.000156,0.000174,2.545668e-08,0.000032,0.000005,0.000118,-0.000002,4.259309e-06,1.162305e-04,0.000015,...,0.000135,7.934634e-06,-0.000007,-4.772409e-07,0.000113,1.404000e-04,0.000233,2.777725e-06,1.639298e-04,-6.338837e-06
1,-0.000077,-0.000126,2.026994e-05,0.000208,0.000020,-0.000092,0.000060,3.185147e-05,8.035413e-05,0.000074,...,0.000192,-2.682939e-05,0.000013,-2.221413e-05,-0.000052,-5.484561e-05,-0.000236,4.927292e-05,2.096190e-05,-2.807977e-05
2,-0.000004,0.000019,-1.068567e-05,-0.000013,-0.000090,-0.000004,0.000003,2.775734e-06,7.193134e-06,-0.000014,...,0.000007,3.268009e-06,0.000011,-1.561348e-05,-0.000002,-7.741583e-07,0.000020,-2.136061e-05,-3.674291e-06,2.438235e-05
3,-0.000011,-0.000011,-3.988167e-07,0.000020,-0.000053,-0.000015,-0.000037,7.418609e-07,-1.795465e-05,-0.000046,...,0.000027,-2.579966e-05,-0.000019,5.111608e-06,-0.000013,8.691600e-08,0.000040,4.068026e-06,1.522878e-05,-7.227746e-05
4,-0.000011,0.000014,-8.518804e-06,-0.000088,-0.000022,0.000010,-0.000056,-1.908289e-05,2.710562e-08,-0.000014,...,0.000004,-4.016718e-05,-0.000072,1.435188e-05,-0.000016,-1.605102e-05,-0.000083,-2.538355e-06,-1.421866e-05,8.210464e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,0.000004,0.000018,-1.079191e-05,0.000007,-0.000008,0.000039,-0.000003,1.349893e-05,-3.321187e-05,0.000012,...,0.000020,-4.572846e-06,-0.000050,5.655049e-07,-0.000037,3.235168e-05,-0.000006,-3.814847e-06,-3.074334e-05,-5.403788e-05
334,-0.000003,-0.000058,2.385363e-06,0.000007,0.000002,-0.000004,0.000002,1.114226e-06,-2.259712e-05,0.000032,...,0.000024,-7.174087e-07,0.000024,-2.046679e-06,-0.000003,3.199453e-05,0.000008,-7.239508e-06,-7.372744e-06,2.436686e-07
335,-0.000001,-0.000026,-3.732483e-05,-0.000032,-0.000041,-0.000015,-0.000047,-4.523514e-05,-3.728120e-05,-0.000183,...,-0.000007,-1.991442e-04,-0.000128,-1.070032e-04,-0.000008,1.797717e-05,-0.000038,-3.420447e-05,-1.883704e-05,-1.055075e-04
336,-0.000002,0.000021,-7.373370e-06,0.000008,-0.000009,-0.000026,-0.000013,3.033330e-06,2.488039e-05,0.000004,...,0.000013,-2.613796e-05,-0.000021,-1.236581e-05,-0.000006,2.622954e-06,0.000004,-3.221886e-07,-4.681422e-07,-1.940611e-05


In [365]:
from statsmodels.tsa.arima.model import ARIMA

In [366]:
ou = np.cumsum(residual)

dt = 1/len(ou)


In [367]:
ou

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1647817320000,-1.089167e-03,3.044079e-04,-5.453197e-06,2.345280e-05,-3.106578e-06,-1.245869e-03,-1.161812e-05,-5.312642e-06,-9.874401e-04,-1.620543e-05,...,-3.121083e-04,1.242569e-04,2.401350e-03,1.310817e-03,-1.094634e-03,-4.955247e-04,3.781282e-04,-9.458435e-06,-2.002293e-04,-6.917725e-06
1647817380000,-1.222937e-03,1.322764e-03,-1.090639e-05,-4.341174e-04,-6.213157e-06,-1.621852e-03,-2.323624e-05,-1.062528e-05,-3.006753e-04,1.295460e-02,...,-1.688462e-03,8.082862e-04,-1.673414e-02,1.428338e-03,-6.938009e-04,-7.480361e-04,5.917636e-05,-1.891687e-05,-4.004910e-04,-1.383545e-05
1647817440000,-2.058298e-03,1.561884e-03,-1.635959e-05,2.252660e-04,1.585577e-03,-1.488519e-03,-3.485436e-05,-1.593793e-05,1.083099e-03,1.293840e-02,...,-1.886907e-03,1.827510e-03,-2.648697e-02,3.544342e-04,-1.674108e-03,-7.572977e-04,-2.748745e-05,-2.837530e-05,-6.007854e-04,-2.075317e-05
1647817500000,-2.830515e-03,2.125085e-03,-2.181279e-05,-1.048021e-03,-9.886670e-06,-2.589021e-03,-4.647248e-05,-2.125057e-05,1.280838e-03,1.292219e-02,...,-1.895064e-03,3.069260e-03,-2.155759e-02,4.866160e-03,-2.712969e-03,-1.253177e-03,-1.508203e-03,-2.019994e-03,-2.597914e-04,-2.767090e-05
1647817560000,-1.942402e-03,2.299193e-03,-2.726599e-05,-1.428650e-03,-1.299325e-05,-2.784169e-03,-5.809060e-05,-2.656321e-05,1.408895e-03,1.290599e-02,...,-2.017415e-03,1.033763e-02,-2.645628e-02,5.545647e-03,-2.311486e-03,-1.992721e-03,-1.594867e-03,-2.029453e-03,-2.796780e-04,-3.458862e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,2.332578e-05,7.592293e-05,2.042803e-05,7.303838e-05,1.163744e-05,5.184800e-05,4.352223e-05,1.990150e-05,4.202364e-05,6.070659e-05,...,3.055652e-05,-4.602910e-05,-1.224445e-05,2.961631e-05,7.921230e-06,3.469457e-05,3.246482e-04,3.543191e-05,7.449652e-05,2.591424e-05
1649026680000,1.478652e-05,4.812855e-05,1.294960e-05,4.630000e-05,7.377130e-06,3.286713e-05,2.758932e-05,1.261583e-05,2.663934e-05,3.848271e-05,...,1.937019e-05,-2.917845e-05,-7.761920e-06,1.877417e-05,5.021373e-06,2.199335e-05,2.057988e-04,2.246076e-05,4.722433e-05,1.642738e-05
1649026740000,8.006326e-06,2.605974e-05,7.011706e-06,2.506965e-05,3.994430e-06,1.779628e-05,1.493855e-05,6.830980e-06,1.442417e-05,2.083690e-05,...,1.048820e-05,-1.579900e-05,-4.202779e-06,1.016549e-05,2.718879e-06,1.190855e-05,1.114321e-04,1.216163e-05,2.557014e-05,8.894791e-06


In [368]:
for i in ou.columns:
    model = ARIMA(ou[i], order=(1, 0, 0)) 
    fittedmodel = model.fit()  

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self.

In [369]:
results = pd.read_html(fittedmodel.summary().tables[1].as_html(),header=None,index_col=0)[0]


In [370]:
var = float(results[1].iloc[3])
a = float(results[1].iloc[1])
b = float(results[1].iloc[2])

In [371]:
kappa = -np.log(b)/dt
m = a/(1-np.exp(-kappa*dt))
sigma = np.sqrt(var*2*kappa/(1-np.exp(-2*kappa*dt)))
sigmaeq = np.sqrt(var/(1-np.exp(-2*kappa*dt)))

In [372]:
period = len(ou) / (6*24*2)

kappa > period

print (kappa)

print (period)

92.9499491628952
70.0


In [373]:
if kappa > period:
    sscore = -m/sigmaeq
    print(sscore)

-178.27549541437125
